# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#!pip install matplotlib
#!pip install pandas
#!pip install requests
#!pip install gmaps


# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_location = "../WeatherPy/output_data/cities.csv"

weather_data = pd.read_csv(data_location)


weather_data

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Tutóia,75,BR,1586029908,83,-2.76,-42.27,84.20,13.87
1,1,Abidjan,75,CI,1586029689,79,5.31,-4.02,84.20,11.41
2,2,Rikitea,45,PF,1586029671,80,-23.12,-134.97,76.28,15.43
3,3,NEDJO,99,ET,1586029908,22,9.50,35.50,67.48,2.48
4,4,Kiama,2,AU,1586029909,58,-34.68,150.87,57.20,13.87
...,...,...,...,...,...,...,...,...,...,...
568,568,Beloha,100,MG,1586029989,75,-25.17,45.05,69.67,17.76
569,569,Kodīnar,0,IN,1586029989,61,20.79,70.70,77.11,11.88
570,570,Bondo,1,CD,1586029989,75,3.81,23.69,73.65,0.83
571,571,Mouila,100,GA,1586029989,90,-1.87,11.06,72.54,1.25


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

locations = weather_data[["Lat", "Lng"]].astype(float)

fig = gmaps.figure()

# Create a poverty Heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=weather_data["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)



fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Criteria
# Max Temp lower than 80, but higher than 70
# Wind Speed less than 10
# Zero Cloudiness


nice_weather = weather_data[weather_data["Max Temp"]<=80]
nice_weather = nice_weather[nice_weather["Max Temp"]>=70]
nice_weather = nice_weather[nice_weather["Wind Speed"]<=10]
nice_weather = nice_weather[nice_weather["Cloudiness"]==0]
nice_weather = nice_weather.dropna()
nice_weather

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
43,43,Port Alfred,0,ZA,1586029672,78,-33.59,26.89,72.00,4.00
64,64,East London,0,ZA,1586029918,50,-33.02,27.91,76.17,8.50
94,94,Tandil,0,AR,1586029922,51,-37.32,-59.13,73.99,5.99
151,151,Chiungutwa,0,TZ,1586029930,88,-10.88,38.98,70.66,0.25
175,175,Makkah al Mukarramah,0,SA,1586029728,40,21.43,39.83,79.56,1.41
192,192,Upington,0,ZA,1586029936,41,-28.45,21.26,77.00,8.05
256,256,Atar,0,MR,1586029945,17,20.52,-13.05,78.01,6.62
339,339,Aswān,0,EG,1586029957,18,24.09,32.91,76.05,5.95
342,342,Rūdbār,0,AF,1586029957,19,30.15,62.60,73.36,3.96
354,354,Puri,0,IN,1586029959,85,19.80,85.85,79.86,8.57


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = nice_weather.copy()

hotel_df.loc[:,"Coordinates"] = hotel_df.loc[:,"Lat"].astype(str) +"," + hotel_df.loc[:,"Lng"].astype(str)

hotel_df



,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Coordinates
43,43,Port Alfred,0,ZA,1586029672,78,-33.59,26.89,72.00,4.00,"-33.59,26.89"
64,64,East London,0,ZA,1586029918,50,-33.02,27.91,76.17,8.50,"-33.02,27.91"
94,94,Tandil,0,AR,1586029922,51,-37.32,-59.13,73.99,5.99,"-37.32,-59.13"
151,151,Chiungutwa,0,TZ,1586029930,88,-10.88,38.98,70.66,0.25,"-10.88,38.98"
175,175,Makkah al Mukarramah,0,SA,1586029728,40,21.43,39.83,79.56,1.41,"21.43,39.83"
192,192,Upington,0,ZA,1586029936,41,-28.45,21.26,77.00,8.05,"-28.45,21.26"
256,256,Atar,0,MR,1586029945,17,20.52,-13.05,78.01,6.62,"20.52,-13.05"
339,339,Aswān,0,EG,1586029957,18,24.09,32.91,76.05,5.95,"24.09,32.91"
342,342,Rūdbār,0,AF,1586029957,19,30.15,62.60,73.36,3.96,"30.15,62.6"
354,354,Puri,0,IN,1586029959,85,19.80,85.85,79.86,8.57,"19.8,85.85"


In [6]:
target_radius = 5000
target_type = "hotel"
target_search = "hotel"

hotel_list = []

for location in hotel_df["Coordinates"]:
    params = {
        "location": location,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    try:
        hotel_list.append(response["results"][0]["name"])
    except:
        hotel_list.append("***None Found***")
        print(f"No hotels within 5000m of {location}, hotel noted as ***None Found***")

hotel_df["Hotel Name"] = hotel_list

hotel_df

No hotels within 5000m of -10.88,38.98, hotel noted as ***None Found***
No hotels within 5000m of 30.15,62.6, hotel noted as ***None Found***


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Coordinates,Hotel Name
43,43,Port Alfred,0,ZA,1586029672,78,-33.59,26.89,72.00,4.00,"-33.59,26.89",The Halyards Hotel
64,64,East London,0,ZA,1586029918,50,-33.02,27.91,76.17,8.50,"-33.02,27.91",Blue Lagoon Hotel
94,94,Tandil,0,AR,1586029922,51,-37.32,-59.13,73.99,5.99,"-37.32,-59.13",Hosteria de la Cascada
151,151,Chiungutwa,0,TZ,1586029930,88,-10.88,38.98,70.66,0.25,"-10.88,38.98",***None Found***
175,175,Makkah al Mukarramah,0,SA,1586029728,40,21.43,39.83,79.56,1.41,"21.43,39.83",Nawazi Watheer Hotel
192,192,Upington,0,ZA,1586029936,41,-28.45,21.26,77.00,8.05,"-28.45,21.26",The Maxton Boutique Hotel Upington
256,256,Atar,0,MR,1586029945,17,20.52,-13.05,78.01,6.62,"20.52,-13.05",Odar kanawal
339,339,Aswān,0,EG,1586029957,18,24.09,32.91,76.05,5.95,"24.09,32.91",Hotel Sofitel Legend Old Cataract Aswan
342,342,Rūdbār,0,AF,1586029957,19,30.15,62.60,73.36,3.96,"30.15,62.6",***None Found***
354,354,Puri,0,IN,1586029959,85,19.80,85.85,79.86,8.57,"19.8,85.85","MAYFAIR Heritage, Puri"


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_template)

fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))